In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical

import os
import random
import shutil
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [3]:
"""
    Split the dataset into training and testing sets.

    Args:
    - directory: The directory containing the dataset.
    - ratio: The ratio of training data to total data (default is 0.8).

    Returns:
    - train_files: List of file paths for training.
    - test_files: List of file paths for testing.
    """
def split_dataset(directory, ratio=0.8):
    # Get all file paths in the directory
    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg'):
                file_path = os.path.join(root, file)
                all_files.append(file_path)

    # Shuffle the file paths
    random.shuffle(all_files)

    # Split the file paths into training and testing sets
    split_idx = int(len(all_files) * ratio)
    train_files = all_files[:split_idx]
    test_files = all_files[split_idx:]

    return train_files, test_files


def combine_subdirectories(main_dir):
    """
    Combines two subdirectories in each subdirectory of the main directory into one.

    Args:
    - main_dir (str): Path to the main directory containing subdirectories.
    """
    # Iterate over each subdirectory
    for subdir in os.listdir(main_dir):
        subdir_path = os.path.join(main_dir, subdir)
        if os.path.isdir(subdir_path):
            # Get the paths of the two subdirectories
            subdirs = [os.path.join(subdir_path, subsubdir) for subsubdir in os.listdir(subdir_path)]
            # Ensure that both paths are directories
            if all(os.path.isdir(path) for path in subdirs):
                # Combine the files from the second subdirectory into the first subdirectory
                for file in os.listdir(subdirs[1]):
                    src = os.path.join(subdirs[1], file)
                    dst = os.path.join(subdirs[0], file)
                    shutil.move(src, dst)
                # Remove the now empty second subdirectory
                os.rmdir(subdirs[1])

In [5]:
# this was a test but i think its useless
directory = "Processed-CASIA-Iris-Interval"
combine_subdirectories(directory)
train_files, test_files = split_dataset(directory, ratio=0.7)

print("Number of training files:", len(train_files))
print("Number of testing files:", len(test_files))

Number of training files: 1847
Number of testing files: 792


In [4]:
num_classes = 411
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output to feed into a dense layer
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

# Output layer
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 478, 638, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 239, 319, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 237, 317, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 118, 158, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 116, 156, 128)     73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 58, 78, 128)      0

In [6]:
# Define image dimensions
img_width, img_height = 640, 480
input_shape = (img_height, img_width, 1)  # Grayscale image

# Define batch size
batch_size = 32

#Define data generator for training data
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

# train_datagen.fit(train_files)

train_generator = train_datagen.flow_from_directory(
    directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10)

Found 2216 images belonging to 249 classes.
Found 423 images belonging to 249 classes.


: 